In [ ]:
from sklearn.model_selection import KFold
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline 
import time, math

tf.reset_default_graph()  
tf.set_random_seed(777)  
xy = np.loadtxt("Input_data_path", delimiter=',', dtype=np.float32) #X_8, X_16

xy_data=xy.reshape(-1,18) #X_16
# xy_data=xy.reshape(-1,10) #X_8

np.random.shuffle(xy_data)

x_data_training = xy_data[:4500, :-2]
y_data_training = xy_data[:4500, -2:]
x_data_test = xy_data[4500:, :-2]
y_data_test = xy_data[4500:, -2:]

X = tf.placeholder(tf.float32, shape=[None, 16]) #X_16
# X = tf.placeholder(tf.float32, shape=[None, 8]) #X_8
Y = tf.placeholder(tf.float32, shape=[None, 2])

#X_16
W1 = tf.get_variable("weight1", shape=[16, 64], initializer=tf.contrib.layers.xavier_initializer())
W2 = tf.get_variable("weight2", shape=[64, 32], initializer=tf.contrib.layers.xavier_initializer())
W3 = tf.get_variable("weight3", shape=[32, 16], initializer=tf.contrib.layers.xavier_initializer())
W4 = tf.get_variable("weight4", shape=[16, 2], initializer=tf.contrib.layers.xavier_initializer())

# #X_8
# W1 = tf.get_variable("weight1", shape=[8, 32], initializer=tf.contrib.layers.xavier_initializer())
# W2 = tf.get_variable("weight2", shape=[32, 64] , initializer=tf.contrib.layers.xavier_initializer())
# W3 = tf.get_variable("weight3", shape=[64, 32], initializer=tf.contrib.layers.xavier_initializer())
# W4 = tf.get_variable("weight4", shape=[32, 2], initializer=tf.contrib.layers.xavier_initializer())

#X_16
b1 = tf.Variable(tf.random_normal([64]), name='bias1')
b2 = tf.Variable(tf.random_normal([32]), name='bias2')
b3 = tf.Variable(tf.random_normal([16]), name='bias3')
b4 = tf.Variable(tf.random_normal([2]), name='bias4')

# #X_8
# b1 = tf.Variable(tf.random_normal([32]), name='bias1')
# b2 = tf.Variable(tf.random_normal([64]), name='bias2')
# b3 = tf.Variable(tf.random_normal([32]), name='bias3')
# b4 = tf.Variable(tf.random_normal([2]), name='bias4')

L2 = tf.nn.relu(tf.matmul(X, W1) + b1)
L3 = tf.nn.relu(tf.matmul(L2, W2) + b2)
L4 = tf.nn.relu(tf.matmul(L3, W3) + b3)
hypothesis = tf.matmul(L4, W4) + b4

cost = tf.reduce_mean(tf.square(hypothesis - Y))
train_op  = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [ ]:
init = tf.global_variables_initializer()

def run_train(sess, train_x, train_y):
    print("\nStart training")
    sess.run(init)
    batch_size = 500
    for epoch in range(4000):
        total_batch = int(train_x.shape[0] / batch_size)
        for i in range(total_batch):
            batch_x = train_x[i*batch_size:(i+1)*batch_size]
            batch_y = train_y[i*batch_size:(i+1)*batch_size]
            _, c = sess.run([train_op, cost], feed_dict={X: batch_x, Y: batch_y})
            if i % 50 == 0:
                print("Epoch #%d step=%d cost=%f" % (epoch, i, c))

def cross_validate(sess, split_size=5):
    results = []
    kf = KFold(n_splits=split_size)
    for train_idx, val_idx in kf.split(x_data_training, y_data_training):
        train_x = x_data_training[train_idx]
        train_y = y_data_training[train_idx]
        val_x = x_data_training[val_idx]
        val_y = y_data_training[val_idx]
        run_train(sess, train_x, train_y)
        results.append(sess.run(cost, feed_dict={X: val_x, Y: val_y}))
    return results
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    result = cross_validate(sess)
    print("Cross-validation result: %s" % result)
    print("Test cost: %f" % sess.run(cost, feed_dict={X: x_data_test, Y: y_data_test}))
    y_predict_tr = sess.run(hypothesis, feed_dict={X: x_data_training})
    y_predict = sess.run(hypothesis, feed_dict={X: x_data_test})
    saver = tf.train.Saver()
    saver.save(sess, 'Checkpoint_path')


In [ ]:
import random

class Solution:
    
    def __init__(self, num_objectives):
        self.num_objectives = num_objectives
        self.objectives = []
        for _ in range(num_objectives):
            self.objectives.append(None)
        self.attributes = []
        self.rank = sys.maxsize
        self.distance = 0.0
        
    def evaluate_solution(self):
        raise NotImplementedError("Solution class have to be implemented.")
    
    def crossover(self, other):
        raise NotImplementedError("Solution class have to be implemented.")
    
    def mutate(self):
        raise NotImplementedError("Solution class have to be implemented.")
    
    def __rshift__(self, other):
        dominates = False
        
        for i in range(len(self.objectives)):
            if self.objectives[i] > other.objectives[i]:
                return False
                
            elif self.objectives[i] < other.objectives[i]:
                dominates = True
        
        return dominates
        
    def __lshift__(self, other):
        return other >> self


def crowded_comparison(s1, s2):
    if s1.rank < s2.rank:
        return 1
    elif s1.rank > s2.rank:
        return -1
    elif s1.distance > s2.distance:
        return 1
    elif s1.distance < s2.distance:
        return -1
    else:
        return 0

class NSGAII:
    current_evaluated_objective = 0
    
    def __init__(self, num_objectives, mutation_rate=0.1, crossover_rate=1.0):
        self.num_objectives = num_objectives
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        random.seed();
        
    def run(self, P, population_size, num_generations):
        for s in P:
            s.evaluate_solution()
        Q = []
        
        for i in range(num_generations):
            print("Iteracao ", i) 
            R = []
            R.extend(P)
            R.extend(Q)            
            fronts = self.fast_nondominated_sort(R)
            del P[:]            
            for front in fronts.values():
                if len(front) == 0:
                    break
                self.crowding_distance_assignment(front);
                P.extend(front)
                
                if len(P) >= population_size:
                    break
            self.sort_crowding(P)
            if len(P) > population_size:
                del P[population_size:]
            Q = self.make_new_pop(P)
            
    def sort_ranking(self, P):
        for i in range(len(P) - 1, -1, -1):
            for j in range(1, i + 1):
                s1 = P[j - 1]
                s2 = P[j]
                if s1.rank > s2.rank:
                    P[j - 1] = s2
                    P[j] = s1
                    
    def sort_objective(self, P, obj_idx):
        for i in range(len(P) - 1, -1, -1):
            for j in range(1, i + 1):
                s1 = P[j - 1]
                s2 = P[j]
                
                if s1.objectives[obj_idx] > s2.objectives[obj_idx]:
                    P[j - 1] = s2
                    P[j] = s1
                    
    def sort_crowding(self, P):
        for i in range(len(P) - 1, -1, -1):
            for j in range(1, i + 1):
                s1 = P[j - 1]
                s2 = P[j]
                
                if crowded_comparison(s1, s2) < 0:
                    P[j - 1] = s2
                    P[j] = s1
                
    def make_new_pop(self, P):
        Q = []
        
        while len(Q) != len(P):
            selected_solutions = [None, None]
            
            while selected_solutions[0] == selected_solutions[1]:
                for i in range(2):
                    s1 = random.choice(P)
                    s2 = s1
                    
                    while s1 == s2:
                        s2 = random.choice(P)
                    
                    if crowded_comparison(s1, s2) > 0:
                        selected_solutions[i] = s1
                        
                    else:
                        selected_solutions[i] = s2
            
            if random.random() < self.crossover_rate:
                child_solution = selected_solutions[0].crossover(selected_solutions[1])
                
                if random.random() < self.mutation_rate:
                    child_solution.mutate()
                    
                child_solution.evaluate_solution()
                
                Q.append(child_solution)
        
        return Q
        
    def fast_nondominated_sort(self, P):
        
        fronts = {}
        
        S = {}
        n = {}
        for s in P:
            S[s] = []
            n[s] = 0
            
        fronts[1] = []
        
        for p in P:
            for q in P:
                if p == q:
                    continue
                
                if p >> q:
                    S[p].append(q)
                
                elif p << q:
                    n[p] += 1
            
            if n[p] == 0:
                fronts[1].append(p)
        
        i = 1
        
        while len(fronts[i]) != 0:
            next_front = []
            
            for r in fronts[i]:
                for s in S[r]:
                    n[s] -= 1
                    if n[s] == 0:
                        next_front.append(s)
            
            i += 1
            fronts[i] = next_front
                    
        return fronts
        
    def crowding_distance_assignment(self, front):
        
        for p in front:
            p.distance = 0
        
        for obj_index in range(self.num_objectives):
            self.sort_objective(front, obj_index)
            
            front[0].distance = float('inf')
            front[len(front) - 1].distance = float('inf')
            
            for i in range(1, len(front) - 1):
                front[i].distance += (front[i + 1].distance - front[i - 1].distance)

In [ ]:
import sys, random, math
import tensorflow as tf
import datetime

class T1Solution(Solution):
    
    def __init__(self):
        Solution.__init__(self, 2)
        
        self.xmin = 0.0
        self.xmax  = 1.0
        
        for _ in range(16): #X_16
#         for _ in range(8): #X_8
            self.attributes.append(random.random())
        self.evaluate_solution()
    
    def evaluate_solution_test(self):
        self.objectives[0] = self.attributes[0]
        self.objectives[1] = self.attributes[1]
   
    def evaluate_solution(self):  
        tf.reset_default_graph()

        X = tf.placeholder(tf.float32, shape=[None, 16]) #X_16
#         X = tf.placeholder(tf.float32, shape=[None, 8]) #X_8
        Y = tf.placeholder(tf.float32, shape=[None, 2])

        #X_16
        W1 = tf.get_variable(name='weight1', shape=[16, 64])
        W2 = tf.get_variable(name='weight2', shape=[64, 32])
        W3 = tf.get_variable(name='weight3', shape=[32, 16])
        W4 = tf.get_variable(name='weight4', shape=[16, 2])
        
#         #X_8
#         W1 = tf.get_variable(name='weight1', shape=[8, 32])
#         W2 = tf.get_variable(name='weight2', shape=[32, 64])
#         W3 = tf.get_variable(name='weight3', shape=[64, 32])
#         W4 = tf.get_variable(name='weight4', shape=[32, 2])        

        #X_16
        b1 = tf.Variable(tf.random_normal([64]), name='bias1')
        b2 = tf.Variable(tf.random_normal([32]), name='bias2')
        b3 = tf.Variable(tf.random_normal([16]), name='bias3')
        b4 = tf.Variable(tf.random_normal([2]), name='bias4')

#         #X_8
#         b1 = tf.Variable(tf.random_normal([32]), name='bias1')
#         b2 = tf.Variable(tf.random_normal([64]), name='bias2')
#         b3 = tf.Variable(tf.random_normal([32]), name='bias3')
#         b4 = tf.Variable(tf.random_normal([2]), name='bias4')
        
        L2 = tf.nn.relu(tf.matmul(X, W1) + b1)
        L3 = tf.nn.relu(tf.matmul(L2, W2) + b2)
        L4 = tf.nn.relu(tf.matmul(L3, W3) + b3)
        hypothesis = tf.matmul(L4, W4) + b4
        
        with tf.Session() as sess:   
            saver = tf.train.Saver()
            saver.restore(sess, "checkpoint_path")
                         
            self.objectives[0] = sess.run(1/hypothesis[:,0], feed_dict={X: [self.attributes]})
            self.objectives[1] = sess.run(1/hypothesis[:,1], feed_dict={X: [self.attributes]})
            
        
    def crossover(self, other):
        child_solution = T1Solution()
        
        for i in range(16): #X_16
#         for i in range(8): #X_8   
            child_solution.attributes[i] = math.sqrt(self.attributes[i] * other.attributes[i])
        return child_solution
    
    def mutate(self):
        self.attributes[random.randint(0, 15)] = random.random() #X_16
#         self.attributes[random.randint(0, 7)] = random.random() #X_8
    
if __name__ == '__main__':
    start = datetime.datetime.now()
    for j in range(20):
        
        nsga2 = NSGAII(2, 0.1, 1.0)
    
        P = []
        for i in range(500):
            P.append(T1Solution())
    
        nsga2.run(P, 50, 200)
    
        csv_file_1 = open("save_path"+"_"+str(j+1)+".csv", 'w')
        csv_file_0 = open("save_path"+"_"+str(j+1)+".csv", 'w')
    
        for i in range(len(P)):
            ys_min= 'YS_minimum vlaue'
            ys_max= 'YS_maximum vlaue'
            uts_min= 'UTS_minimum vlaue'
            uts_max= 'UTS_maximum vlaue'
            if (1/P[i].objectives[0]*(ys_max-ys_min))+ys_min>0.9*(ys_max-ys_min)+ys_min and (1/P[i].objectives[1]*(uts_max-uts_min))+uts_min>0.9*(uts_max-uts_min)+uts_min and (1/P[i].objectives[0]*(ys_max-ys_min))+ys_min<(1/P[i].objectives[1]*(uts_max-uts_min))+uts_min:
                csv_file_1.write("" + str(1/P[i].objectives[0]) + ", " + str(1/P[i].objectives[1]) + "\n")
                csv_file_0.write(("" + str(P[i].attributes[0]) + ", " + str(P[i].attributes[1]) + ", " + str(P[i].attributes[2]) + ", " + str(P[i].attributes[3])+ ", " +str(P[i].attributes[4]) + ", " + str(P[i].attributes[5])+ ", " + str(P[i].attributes[6])+ ", " + str(P[i].attributes[7]) + ", " + str(P[i].attributes[8])+ ", " +str(P[i].attributes[9]) + ", " + str(P[i].attributes[10])+ ", " +str(P[i].attributes[11]) + ", " + str(P[i].attributes[12])+ ", " +str(P[i].attributes[13]) + ", " + str(P[i].attributes[14])+", " +str(P[i].attributes[15]))+"\n")  #X_16
#                 csv_file_0.write("" + str(P[i].attributes[0]) + ", " + str(P[i].attributes[1]) + ", " + str(P[i].attributes[2]) + ", " + str(P[i].attributes[3])+ ", " +str(P[i].attributes[4]) + ", " + str(P[i].attributes[5])+ ", " + str(P[i].attributes[6])+ ", " + str(P[i].attributes[7])+"\n") #X_8

        csv_file_1.close()
        csv_file_0.close()
    stop = datetime.datetime.now()    
print('\n Ended at', stop, 'Elaspsed Time =' , stop-start, '\n')